In [13]:
# Inspect the renderCV dictionary structure
import yaml
import json

print("=== RenderCV Dictionary Structure ===")
print(f"Top-level keys: {list(rendercv_dict.keys())}")

if 'cv' in rendercv_dict:
    cv_data = rendercv_dict['cv']
    print(f"\nCV section keys: {list(cv_data.keys())}")
    
    # Check sections
    if 'sections' in cv_data:
        print(f"\nSections: {list(cv_data['sections'].keys())}")
        for section_name, section_data in cv_data['sections'].items():
            if isinstance(section_data, list):
                print(f"  {section_name}: {len(section_data)} items")
            else:
                print(f"  {section_name}: {type(section_data)}")

print(f"\nDesign section: {rendercv_dict.get('design', 'Missing')}")

# Save to YAML to inspect
with open('debug_rendercv.yaml', 'w') as f:
    yaml.dump(rendercv_dict, f, default_flow_style=False)
print(f"\n✅ Saved renderCV dict to debug_rendercv.yaml for inspection")

=== RenderCV Dictionary Structure ===
Top-level keys: ['cv']

CV section keys: ['name', 'location', 'email', 'phone', 'website', 'social_networks', 'sections']

Sections: ['summary', 'education', 'experience', 'publications', 'projects', 'technologies']
  summary: 1 items
  education: 4 items
  experience: 7 items
  publications: 2 items
  projects: 1 items
  technologies: 2 items

Design section: Missing

✅ Saved renderCV dict to debug_rendercv.yaml for inspection


In [15]:
# Add the missing design section and test PDF generation
test_rendercv_dict = dict(rendercv_dict)  # Copy

# Add the design section that renderCV requires
test_rendercv_dict['design'] = {
    'theme': 'classic',
    'font_size': '10pt',
    'page_size': 'letterpaper'
}

print(f"Added design section: {test_rendercv_dict['design']}")

# Test PDF generation
output_file = Path("fixed_resume.pdf")

try:
    rendercv.api.create_a_pdf_from_a_python_dictionary(test_rendercv_dict, str(output_file))
    
    if output_file.exists():
        file_size = output_file.stat().st_size
        print(f"✅ PDF created successfully: {file_size} bytes")
        
        # Read the PDF
        with open(output_file, 'rb') as f:
            pdf_data = f.read()
        print(f"✅ PDF data read: {len(pdf_data)} bytes")
        
        print("🎉 SUCCESS! The renderCV pipeline is now working!")
        
    else:
        print("❌ PDF file was still not created")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

Added design section: {'theme': 'classic', 'font_size': '10pt', 'page_size': 'letterpaper'}
❌ PDF file was still not created


# Scrap: Real example

In [1]:
import yaml 

def load_yaml(yaml_path: str):
    """
    Loads a YAML file using PyYAML.
    This works in Python 3.7+ because dicts preserve insertion order.
    """
    with open(yaml_path, 'r') as file:
        # Use safe_load for security reasons.
        return yaml.safe_load(file)

def dump_yaml(d: dict, yaml_path: str):
    """
    Dumps a dictionary to a YAML file using PyYAML,
    preserving key order and using a readable block style.
    """
    with open(yaml_path, 'w') as file:
        # sort_keys=False is crucial to prevent alphabetical sorting.
        # default_flow_style=False ensures a readable, multi-line format.
        yaml.dump(d, file, sort_keys=False, default_flow_style=False)

In [2]:
import json 
import os 

from hired.util import proj_files
from hired import mk_resume

resume_json_path = proj_files / 'tests/fixtures/real_example_resume.json'
content = json.load(open(resume_json_path))

print(resume_json_path)

/Users/thorwhalen/Dropbox/py/proj/t/hired/hired/tests/fixtures/real_example_resume.json


In [3]:
# convert content to yaml
import yaml
resume_yaml_path = proj_files / 'tests/fixtures/real_example_resume.yaml'
dump_yaml(content, resume_yaml_path)
print(resume_yaml_path)

/Users/thorwhalen/Dropbox/py/proj/t/hired/hired/tests/fixtures/real_example_resume.yaml


In [4]:
from hired.renderers.w_rendercv import *

# Convert and render
rendercv_dict = resumejson_to_rendercv(content)
pdf_bytes = render_resume_w_rendercv(rendercv_dict, 'pdf', theme='classic')

# Save to file
with open('resume.pdf', 'wb') as f:
    f.write(pdf_bytes)

ValueError: Conversion failed: Traceback (most recent call last):
  File "/Users/thorwhalen/.pyenv/versions/3.10.13/envs/p10/bin/jsonresume_to_rendercv", line 7, in <module>
    sys.exit(main())
  File "/Users/thorwhalen/.pyenv/versions/p10/lib/python3.10/site-packages/jsonresume_to_rendercv/converter.py", line 197, in main
    convert(input_file, output_file)
  File "/Users/thorwhalen/.pyenv/versions/p10/lib/python3.10/site-packages/jsonresume_to_rendercv/converter.py", line 176, in convert
    converter = JSONResumeConverter(json_resume)
  File "/Users/thorwhalen/.pyenv/versions/p10/lib/python3.10/site-packages/jsonresume_to_rendercv/converter.py", line 51, in __init__
    "education": self.format_education(json_resume.get("education", [])),
  File "/Users/thorwhalen/.pyenv/versions/p10/lib/python3.10/site-packages/jsonresume_to_rendercv/converter.py", line 88, in format_education
    return [
  File "/Users/thorwhalen/.pyenv/versions/p10/lib/python3.10/site-packages/jsonresume_to_re

In [16]:
pwd

'/Users/thorwhalen/Dropbox/py/proj/t/hired/misc'

In [10]:
# Test the improved function with normalization
from hired.renderers.w_rendercv import resumejson_to_rendercv, render_resume_w_rendercv

try:
    print("Testing conversion with normalized data...")
    rendercv_dict = resumejson_to_rendercv(content)
    print("✅ Conversion successful!")
    print(f"RenderCV dict keys: {list(rendercv_dict.keys())}")
    
    # Try to render as PDF
    print("Rendering PDF...")
    pdf_bytes = render_resume_w_rendercv(rendercv_dict, 'pdf', theme='classic')
    print(f"✅ PDF rendered: {len(pdf_bytes)} bytes")
    
    # Save to file
    with open('resume_normalized.pdf', 'wb') as f:
        f.write(pdf_bytes)
    print("✅ PDF saved as 'resume_normalized.pdf'")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

Testing conversion with normalized data...
✅ Conversion successful!
RenderCV dict keys: ['cv']
Rendering PDF...
✅ Conversion successful!
RenderCV dict keys: ['cv']
Rendering PDF...
❌ Error: Output file was not created: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpwewh29nf/resume.pdf
❌ Error: Output file was not created: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpwewh29nf/resume.pdf


Traceback (most recent call last):
  File "/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_12906/1284023449.py", line 12, in <module>
    pdf_bytes = render_resume_w_rendercv(rendercv_dict, 'pdf', theme='classic')
  File "/Users/thorwhalen/Dropbox/py/proj/t/hired/hired/renderers/w_rendercv.py", line 590, in render_resume_w_rendercv
    raise ValueError(f"Output file was not created: {output_file}")
ValueError: Output file was not created: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpwewh29nf/resume.pdf


In [7]:
# Check available methods in rendercv.api
import rendercv.api
print("Available methods in rendercv.api:")
methods = [attr for attr in dir(rendercv.api) if not attr.startswith('_')]
for method in methods:
    print(f"  {method}")
    
# Also test just the conversion part to confirm it works
print("\n🎉 JSON Resume to renderCV conversion is working!")
print(f"Converted data has these keys: {list(rendercv_dict.keys())}")
if 'cv' in rendercv_dict:
    print(f"CV section has: {list(rendercv_dict['cv'].keys())}")

Available methods in rendercv.api:
  create_a_markdown_file_from_a_python_dictionary
  create_a_markdown_file_from_a_yaml_string
  create_a_pdf_from_a_python_dictionary
  create_a_pdf_from_a_yaml_string
  create_a_typst_file_from_a_python_dictionary
  create_a_typst_file_from_a_yaml_string
  create_an_html_file_from_a_python_dictionary
  create_an_html_file_from_a_yaml_string
  create_contents_of_a_markdown_file_from_a_python_dictionary
  create_contents_of_a_markdown_file_from_a_yaml_string
  create_contents_of_a_typst_file_from_a_python_dictionary
  create_contents_of_a_typst_file_from_a_yaml_string
  functions
  read_a_python_dictionary_and_return_a_data_model
  read_a_yaml_string_and_return_a_data_model

🎉 JSON Resume to renderCV conversion is working!
Converted data has these keys: ['cv']
CV section has: ['name', 'location', 'email', 'phone', 'website', 'social_networks', 'sections']


In [9]:
# Test just the renderCV PDF generation directly
import tempfile
from pathlib import Path

try:
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_path = Path(temp_dir)
        output_file = temp_path / "test_resume.pdf"
        
        print(f"Attempting to create PDF at: {output_file}")
        print(f"RenderCV dict type: {type(rendercv_dict)}")
        print(f"RenderCV dict keys: {list(rendercv_dict.keys()) if isinstance(rendercv_dict, dict) else 'Not a dict'}")
        
        # Try the renderCV API call directly
        rendercv.api.create_a_pdf_from_a_python_dictionary(rendercv_dict, str(output_file))
        
        print(f"PDF file exists: {output_file.exists()}")
        if output_file.exists():
            print(f"PDF file size: {output_file.stat().st_size} bytes")
        else:
            print("PDF file was not created")
            # List what files were created
            files = list(temp_path.glob("*"))
            print(f"Files in temp directory: {files}")
            
except Exception as e:
    print(f"Error during PDF creation: {e}")
    import traceback
    traceback.print_exc()

Attempting to create PDF at: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmprfmvpjwa/test_resume.pdf
RenderCV dict type: <class 'dict'>
RenderCV dict keys: ['cv']
PDF file exists: True
PDF file size: 20889 bytes


In [11]:
# Test with a simpler approach - create a simple wrapper
import tempfile
from pathlib import Path

def simple_render_pdf(rendercv_dict):
    """Simple PDF renderer without all the extra logic"""
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_path = Path(temp_dir)
        output_file = temp_path / "resume.pdf"
        
        # Add basic design if missing
        if 'design' not in rendercv_dict:
            rendercv_dict = dict(rendercv_dict)  # Don't modify original
            rendercv_dict['design'] = {
                'theme': 'classic',
                'font_size': '10pt',
                'page_size': 'letterpaper'
            }
        
        print(f"Attempting to create PDF with design: {rendercv_dict.get('design', 'None')}")
        
        # Create PDF
        rendercv.api.create_a_pdf_from_a_python_dictionary(rendercv_dict, str(output_file))
        
        if output_file.exists():
            with open(output_file, 'rb') as f:
                return f.read()
        else:
            raise ValueError(f"PDF file was not created at {output_file}")

# Test it
try:
    pdf_bytes = simple_render_pdf(rendercv_dict)
    print(f"✅ Simple PDF render successful: {len(pdf_bytes)} bytes")
    
    # Save to file
    with open('resume_simple.pdf', 'wb') as f:
        f.write(pdf_bytes)
    print("✅ PDF saved as 'resume_simple.pdf'")
    
except Exception as e:
    print(f"❌ Simple render failed: {e}")
    import traceback
    traceback.print_exc()

Attempting to create PDF with design: {'theme': 'classic', 'font_size': '10pt', 'page_size': 'letterpaper'}
❌ Simple render failed: PDF file was not created at /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpuyece4b9/resume.pdf


Traceback (most recent call last):
  File "/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_12906/2347455480.py", line 33, in <module>
    pdf_bytes = simple_render_pdf(rendercv_dict)
  File "/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_12906/2347455480.py", line 29, in simple_render_pdf
    raise ValueError(f"PDF file was not created at {output_file}")
ValueError: PDF file was not created at /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpuyece4b9/resume.pdf


In [12]:
# Debug: Try without temporary directory first
output_file = Path("debug_resume.pdf")

# Add basic design if missing  
test_dict = dict(rendercv_dict)  # Don't modify original
if 'design' not in test_dict:
    test_dict['design'] = {
        'theme': 'classic',
        'font_size': '10pt', 
        'page_size': 'letterpaper'
    }

print(f"Attempting to create PDF directly: {output_file}")
print(f"Working directory: {Path.cwd()}")

try:
    rendercv.api.create_a_pdf_from_a_python_dictionary(test_dict, str(output_file))
    
    if output_file.exists():
        print(f"✅ PDF created successfully: {output_file.stat().st_size} bytes")
        
        # Read and report success
        with open(output_file, 'rb') as f:
            pdf_data = f.read()
        print(f"✅ Read {len(pdf_data)} bytes from PDF")
        
        # Clean up
        output_file.unlink()
        print("✅ Cleaned up debug file")
        
    else:
        print("❌ PDF file was not created")
        
        # Check what files were created
        current_files = [f for f in Path.cwd().glob("*") if f.is_file()]
        print(f"Files in current directory: {[f.name for f in current_files]}")
        
except Exception as e:
    print(f"❌ Error during PDF creation: {e}")
    import traceback
    traceback.print_exc()

Attempting to create PDF directly: debug_resume.pdf
Working directory: /Users/thorwhalen/Dropbox/py/proj/t/hired/misc
❌ PDF file was not created
Files in current directory: ['hired_wip.ipynb']
